# CycleGAN Demo: Horse ↔ Zebra Translation

This notebook demonstrates how to train and evaluate a CycleGAN model on unpaired image-to-image translation using the Horse ↔ Zebra dataset.

**Highlights:**
- Unpaired image translation with CycleGAN
- Instance normalization and PatchGAN discriminator
- Training monitoring with loss and gradient plots
- Visual evaluation on test set

----
Imports

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import time
from IPython import display

from models import CycleGAN_Generator, CycleGAN_Discriminator
from utils import train_model, plot_training_metrics, evaluate_on_testset
from dataset import train_horses, train_zebras, test_horses, test_zebras


## Seed and Hyperparameters


In [ ]:
# For reproducibility
seed = 42
keras.backend.clear_session()
tf.random.set_seed(seed)
np.random.seed(seed)

# Hyperparameters
LAMBDA = 10
NUM_EPOCHS = 20
LEARNING_RATE = 2e-4
BETA_1 = 0.5


## Model Instantiation

In [ ]:
# Instantiate Generators
gen_A2B = CycleGAN_Generator(input_image_size=(256, 256, 3), norm_type='instancenorm')
gen_B2A = CycleGAN_Generator(input_image_size=(256, 256, 3), norm_type='instancenorm')
gen_A2B_optimizer = keras.optimizers.Adam(LEARNING_RATE, BETA_1)
gen_B2A_optimizer = keras.optimizers.Adam(LEARNING_RATE, BETA_1)

# Instantiate Discriminators
disc_A = CycleGAN_Discriminator(norm_type='instancenorm')
disc_B = CycleGAN_Discriminator(norm_type='instancenorm')
disc_A_optimizer = keras.optimizers.Adam(LEARNING_RATE, BETA_1)
disc_B_optimizer = keras.optimizers.Adam(LEARNING_RATE, BETA_1)


## Train the CycleGAN model


In [ ]:
results = train_model(
    gen_A2B, gen_B2A,
    gen_A2B_optimizer, gen_B2A_optimizer,
    disc_A, disc_B,
    disc_A_optimizer, disc_B_optimizer,
    train_horses, train_zebras,
    num_epochs=NUM_EPOCHS,
    clip_norm=False,
    max_norm=1.0,
    LAMBDA=LAMBDA
)

total_gen_1_loss, total_gen_2_loss, total_disc_1_loss, total_disc_2_loss, \
gen_1_grad_norms, gen_2_grad_norms, disc_1_grad_norms, disc_2_grad_norms = results


## Training Metrics and curves


In [ ]:
training_metrics = {
    "gen_A2B_loss": total_gen_1_loss,
    "gen_B2A_loss": total_gen_2_loss,
    "disc_A_loss": total_disc_1_loss,
    "disc_B_loss": total_disc_2_loss,
    "gen_A2B_grad_norm": gen_1_grad_norms,
    "gen_B2A_grad_norm": gen_2_grad_norms,
    "disc_A_grad_norm": disc_1_grad_norms,
    "disc_B_grad_norm": disc_2_grad_norms
}

plot_training_metrics(training_metrics)


## Evaluate Horse -> Zebra

In [ ]:
evaluate_on_testset(gen_A2B, test_horses, direction='Horse to Zebra', save_images=False)


# Evaluate Zebra -> Horse

In [ ]:
evaluate_on_testset(gen_B2A, test_zebras, direction='Zebra to Horse', save_images=False)


## Conclusion

In this demo, we trained a CycleGAN model to translate between horses and zebras using unpaired image datasets.

**What we did:**
- Built custom generators and discriminators
- Trained the model with cycle consistency and identity loss
- Visualized training progress
- Evaluated the model on test images

**Next steps you can explore:**
- Train for longer (e.g., 60–100 epochs)
- Add model checkpointing and learning rate schedulers
- Apply to other unpaired datasets

Thanks for trying out this demo!
